In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import itertools
import copy
import os
import sys

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,7)

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.autograd import Variable
from math import sqrt

if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

print(device)

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path) 
from train_utils import *

cuda:0


# Settings

In [2]:
experiment_name = "Covid-19 Cases"

In [3]:
n_experiments = 10
num_epochs = 2000
learning_rate = 0.001

transformation = Identity()  # We do not apply any log, differencing, etc.

models = ['PINPOINT_1CONV', 'PINPOINT_2CONV', 'Naive', 'ARIMA', 'ARIMA_PP', 'Prophet']
seq_length = 14 #2 * seasonality
forecast_horizons = [1, 3, 7]

# Datasets
A recap on the time-series we will use in the followings.

In [4]:
# covid daily cases
df = pd.read_csv("../data/Covid19-italy.csv", parse_dates=["Date"], index_col="Date")
df = df.loc[:pd.Timestamp("2021-12-01"), 'Daily cases']
df.index.freq = 'D'
entire_ts = df

train = entire_ts.loc[:pd.Timestamp("2021-01-14")]
validation_length = int(0.05 * len(train))
validation = entire_ts.loc[train.index[-1] + entire_ts.index.freq:train.index[-1] + validation_length * entire_ts.index.freq]
test = entire_ts.loc[validation.index[-1] + entire_ts.index.freq:]

plot_name = "Covid19"
yaxis_name = "Cases"

print(train)
print(validation)
print(test)

<function concat at 0x7f5838f45940>
Date
2021-01-15    16146
2021-01-16    16310
2021-01-17    12545
2021-01-18     8825
2021-01-19    10497
2021-01-20    13571
2021-01-21    14078
2021-01-22    13633
2021-01-23    13331
2021-01-24    11629
2021-01-25     8562
2021-01-26    10593
2021-01-27    15204
2021-01-28    14372
2021-01-29    13574
2021-01-30    12715
Freq: D, Name: Daily cases, dtype: int64
Date
2021-01-31    11252
2021-02-01     7925
2021-02-02     9660
2021-02-03    13189
2021-02-04    13659
              ...  
2021-11-27    12877
2021-11-28    12932
2021-11-29     7975
2021-11-30    12764
2021-12-01    15085
Freq: D, Name: Daily cases, Length: 305, dtype: int64


# Model

In [5]:
class Square(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 2)

class Cube(torch.nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, t):
        return torch.pow(t, 3)
    
class Printer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, t):
        # print(t)
        print(t.shape)
        return t


class PINPOINT_1CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_1CONV, self).__init__()

        n_kernels_1 = 32
        kernel_size_1 = 3
        out_conv_1 = n_kernels_1 * (input_size - kernel_size_1 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_1, int(out_conv_1/2)), #use without avgpool
            # nn.Linear(int(out_conv_1/2), output_horizon)   
            nn.Linear(int(out_conv_1/2), int(out_conv_1/4)),
            nn.Linear(int(out_conv_1/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_1CONV"

    
class PINPOINT_2CONV(nn.Module):
    def __init__(self, input_size, output_horizon):
        super(PINPOINT_2CONV, self).__init__()
        
        n_kernels_1 = 16
        n_kernels_2 = 32
        kernel_size_1 = 5
        kernel_size_2 = 3
        
        out_conv_1 = input_size - kernel_size_1 + 1
        out_conv_2 = n_kernels_2 * (out_conv_1 - kernel_size_2 + 1)

        self.main = nn.Sequential(           
            nn.Conv1d(in_channels=1, out_channels=n_kernels_1, kernel_size=kernel_size_1),
            Square(),
            nn.Conv1d(in_channels=n_kernels_1, out_channels=n_kernels_2, kernel_size=kernel_size_2),
            Square(),
            nn.Flatten(),      
            
            nn.Linear(out_conv_2, int(out_conv_2/2)), #use without avgpool
            # nn.Linear(int(out_conv_2/4), output_horizon)   
            nn.Linear(int(out_conv_2/2), int(out_conv_2/4)),
            nn.Linear(int(out_conv_2/4), output_horizon)   
        )

    def forward(self, x):
        out = self.main(x)
        return out
    
    def __str__(self):
        return "PINPOINT_2CONV"

In [6]:
names = [f"{m} ({seq_length})" for m in models if m not in ['Naive', 'ARIMA', 'Prophet']]

In [7]:
names.extend(['Naive', 'ARIMA', 'ARIMA_PP', 'Prophet'])

In [8]:
names

['PINPOINT_1CONV (14)',
 'PINPOINT_2CONV (14)',
 'ARIMA_PP (14)',
 'Naive',
 'ARIMA',
 'ARIMA_PP',
 'Prophet']

In [9]:
metrics = pd.DataFrame(columns = forecast_horizons, 
                       index = pd.MultiIndex.from_product([names, ['MAE', 'RMSE']], names=["Model", "Metric"]))

In [10]:
metrics

1    3    7
Model               Metric               
PINPOINT_1CONV (14) MAE     NaN  NaN  NaN
                    RMSE    NaN  NaN  NaN
PINPOINT_2CONV (14) MAE     NaN  NaN  NaN
                    RMSE    NaN  NaN  NaN
ARIMA_PP (14)       MAE     NaN  NaN  NaN
                    RMSE    NaN  NaN  NaN
Naive               MAE     NaN  NaN  NaN
                    RMSE    NaN  NaN  NaN
ARIMA               MAE     NaN  NaN  NaN
                    RMSE    NaN  NaN  NaN
ARIMA_PP            MAE     NaN  NaN  NaN
                    RMSE    NaN  NaN  NaN
Prophet             MAE     NaN  NaN  NaN
                    RMSE    NaN  NaN  NaN

In [11]:
final_results = {}

In [12]:
for forecast_horizon in forecast_horizons:
    final_results[forecast_horizon] = {}
    
    for model_class in models:
        if model_class == 'ARIMA':
            arima_forecast = np.array([])

            _train = train.copy().append(validation.copy())
            _train = transformation.apply(_train)
            _test = transformation.apply(test.copy())
            
            arima_model = ARIMA_model()
            arima_model.train(_train)
            
            for i in range(0, int(len(_test) / forecast_horizon) + 1):
                arima_forecast = np.append(arima_forecast, arima_model.get_forecast(_train, forecast_horizon))

                for j in range(0, forecast_horizon):
                    if len(_test) != 0:
                        _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
                        _test = _test.iloc[1:]
            
            arima_forecast = transformation.inverse(pd.Series(data=arima_forecast[:len(test)], index=test.index))

            final_results[forecast_horizon][f"forecast_ARIMA"] = arima_forecast
            final_results[forecast_horizon][f"mae_ARIMA"] = mean_absolute_error(test, arima_forecast)
            final_results[forecast_horizon][f"rmse_ARIMA"] = sqrt(mean_squared_error(test, arima_forecast))
            metrics.loc[('ARIMA', 'MAE'), forecast_horizon] = mean_absolute_error(test, arima_forecast)
            metrics.loc[('ARIMA', 'RMSE'), forecast_horizon] = sqrt(mean_squared_error(test, arima_forecast))
        
        elif model_class == 'ARIMA_PP':
            arima_forecast = np.array([])

            _train = train.copy().append(validation.copy())
            _train = transformation.apply(_train)
            _test = transformation.apply(test.copy())
            
            arima_model = ARIMA_model()
            arima_model.train(_train)
            
            for i in range(0, int(len(_test) / forecast_horizon) + 1):
                arima_forecast = np.append(arima_forecast, arima_model.get_forecast(_train, forecast_horizon, update=False))

                for j in range(0, forecast_horizon):
                    if len(_test) != 0:
                        _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
                        _test = _test.iloc[1:]
            
            arima_forecast = transformation.inverse(pd.Series(data=arima_forecast[:len(test)], index=test.index))

            final_results[forecast_horizon][f"forecast_ARIMA_PP"] = arima_forecast
            final_results[forecast_horizon][f"mae_ARIMA_PP"] = mean_absolute_error(test, arima_forecast)
            final_results[forecast_horizon][f"rmse_ARIMA_PP"] = sqrt(mean_squared_error(test, arima_forecast))
            metrics.loc[('ARIMA_PP', 'MAE'), forecast_horizon] = mean_absolute_error(test, arima_forecast)
            metrics.loc[('ARIMA_PP', 'RMSE'), forecast_horizon] = sqrt(mean_squared_error(test, arima_forecast))
        
        elif model_class == 'Prophet':
            prophet_forecast = np.array([])
            
            _train = train.copy().append(validation.copy())
            _train = transformation.apply(_train)
            _test = transformation.apply(test.copy())

            for i in range(0, int(len(_test) / forecast_horizon) + 1):
                prophet_forecast = np.append(prophet_forecast, get_prophet_forecast(_train, forecast_horizon).values)

                for j in range(0, forecast_horizon):
                    if len(_test) != 0:
                        _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
                        _test = _test.iloc[1:]

            prophet_forecast = transformation.inverse(pd.Series(data=prophet_forecast[:len(test)], index=test.index))

            final_results[forecast_horizon][f"forecast_Prophet"] = prophet_forecast
            final_results[forecast_horizon][f"mae_Prophet"] = mean_absolute_error(test, prophet_forecast)
            final_results[forecast_horizon][f"rmse_Prophet"] = sqrt(mean_squared_error(test, prophet_forecast))
            metrics.loc[('Prophet', 'MAE'), forecast_horizon] = mean_absolute_error(test, prophet_forecast)
            metrics.loc[('Prophet', 'RMSE'), forecast_horizon] = sqrt(mean_squared_error(test, prophet_forecast))
        
        elif model_class == 'Naive':
            naive_forecast = np.array([])
            
            _train = train.copy().append(validation.copy())
            _train = transformation.apply(_train)
            _test = transformation.apply(test.copy())
            
            for i in range(0, int(len(_test) / forecast_horizon) + 1):

                for j in range(0, forecast_horizon):
                    naive_forecast = np.append(naive_forecast, _train[-1])

                for j in range(0, forecast_horizon):
                    if len(_test) != 0:
                        _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
                        _test = _test.iloc[1:]

            naive_forecast = transformation.inverse(pd.Series(data=naive_forecast[:len(test)], index=test.index))

            final_results[forecast_horizon][f"forecast_Naive"] = naive_forecast
            final_results[forecast_horizon][f"mae_Naive"] = mean_absolute_error(test, naive_forecast)
            final_results[forecast_horizon][f"rmse_Naive"] = sqrt(mean_squared_error(test, naive_forecast))
            metrics.loc[('Naive', 'MAE'), forecast_horizon] = mean_absolute_error(test, naive_forecast)
            metrics.loc[('Naive', 'RMSE'), forecast_horizon] = sqrt(mean_squared_error(test, naive_forecast))
        
        else:
            matches = { 'PINPOINT_1CONV': PINPOINT_1CONV, 
                        'PINPOINT_2CONV': PINPOINT_2CONV}
            
            model_name = model_class
            model_class = matches[model_class]
            
            forecast_maes = []
            forecast_rmses = []

            for i in tqdm(range(0, n_experiments)):
                forecast = np.array([])

                _train = transformation.apply(train.copy()).values
                _validation = transformation.apply(validation.copy()).values   
                _test = transformation.apply(test.copy()).values

                model = model_class(seq_length, forecast_horizon)
                model, scaler = train_model(model, _train, _validation, num_epochs, learning_rate, seq_length, forecast_horizon)
                torch.save(model, f"models/{experiment_name}_{forecast_horizon}_{model_name}.pt")

                # Compute the forecast on the testing set
                _train = train.copy().append(validation.copy())
                _train = transformation.apply(_train)
                _test = transformation.apply(test.copy())

                for i in range(0, int(len(_test) / forecast_horizon) + 1):
                    model.eval()

                    inputs = _train.values.reshape(len(_train), 1)

                    inputs_normalized = scaler.transform(inputs)
                    inputs_normalized = torch.FloatTensor(inputs_normalized[-seq_length:]).to(device)

                    predict = model(inputs_normalized.reshape(1, 1, seq_length))
                    predict = scaler.inverse_transform(predict.cpu().detach().numpy())
                    forecast = np.append(forecast, predict)

                    for j in range(0, forecast_horizon):
                        if len(_test) > 0:
                            _train[_train.index[-1] + train.index.freq] = _test.iloc[0]
                            _test = _test.iloc[1:]

                forecast = transformation.inverse(pd.Series(data=forecast[:len(test)], index=test.index))

                forecast_maes.append(mean_absolute_error(test, forecast))
                forecast_rmses.append(sqrt(mean_squared_error(test, forecast)))

            final_results[forecast_horizon][f"forecast_{model}_{seq_length}"] = forecast
            final_results[forecast_horizon][f"mae_{model}_{seq_length}"] = np.mean(forecast_maes)
            final_results[forecast_horizon][f"rmse_{model}_{seq_length}"] = np.mean(forecast_rmses)
            metrics.loc[(f"{model} ({seq_length})", 'MAE'), forecast_horizon] = np.mean(forecast_maes)
            metrics.loc[(f"{model} ({seq_length})", 'RMSE'), forecast_horizon] = np.mean(forecast_rmses)

  0%|                                                                      | 0/1 [00:00<?, ?it/s]


AttributeError: 'function' object has no attribute 'copy'

In [ ]:
for forecast_horizon in forecast_horizons:
    final_result = final_results[forecast_horizon]
    
    forecasts = {}
    maes = {}
    rmses = {}
    
    for model_class in models:
        if model_class in ['ARIMA', 'ARIMA_PP', 'Prophet', 'Naive']:
            forecasts[f"{model_class}"] = final_result[f"forecast_{model_class}"]
            maes[f"{model_class}"] = final_result[f"mae_{model_class}"]
            rmses[f"{model_class}"] = final_result[f"rmse_{model_class}"]
        else:
            forecasts[f"{model_class}_{seq_length}"] = final_result[f"forecast_{model_class}_{seq_length}"]
            maes[f"{model_class}_{seq_length}"] = final_result[f"mae_{model_class}_{seq_length}"]
            rmses[f"{model_class}_{seq_length}"] = final_result[f"rmse_{model_class}_{seq_length}"]
   
    print(f"Forecast horizon: {forecast_horizon}")
    
    [print(f"MAE of model {model_class}: {x}") for model_class, x in maes.items()]
    [print(f"RMSE of model {model_class}: {x}") for model_class, x in rmses.items()]
    
    print("Plot and a comparison...")

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=train.append(validation).index, 
                             y=train.append(validation),
                             name='Starting train'))

    fig.add_trace(go.Scatter(x=test.index, 
                             y=test,
                             name='Test'))

    [fig.add_trace(go.Scatter(x=forecast.index, 
                              y=forecast,
                              name=f'Predicted points ({model_class})')) for model_class, forecast in forecasts.items()]

    fig.update_layout(title_text = plot_name)
    fig.update_xaxes(title_text = "Date")
    fig.update_yaxes(title_text = yaxis_name)
    fig.show()


In [ ]:
final_result

In [ ]:
for column in metrics.columns:
    print(metrics.loc[metrics.loc[:, column] == min(metrics.loc[:, column])])

In [ ]:
final_csv = metrics.loc[([f"PINPOINT_1CONV ({seq_length})", f"PINPOINT_2CONV ({seq_length})", 
              'Prophet', 'ARIMA', 'ARIMA_PP', 'Naive']), :].astype("float64").round(2)

final_csv.columns = [f"{experiment_name}_{x}" for x in final_csv.columns]
final_csv = final_csv.rename(index={f"PINPOINT_1CONV ({seq_length})": 'PINPOINT_1CONV', 
                                    f"PINPOINT_2CONV ({seq_length})": 'PINPOINT_2CONV'})

final_csv.to_csv(f"results/{experiment_name}.csv")